In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [12]:
X_train

,age,gender,fever,cough,city
27,33,Female,102.0,Strong,Delhi
57,49,Female,99.0,Strong,Bangalore
68,54,Female,104.0,Strong,Kolkata
98,5,Female,98.0,Strong,Mumbai
10,75,Female,NaN,Mild,Delhi
...,...,...,...,...,...
94,79,Male,NaN,Strong,Kolkata
37,55,Male,100.0,Mild,Kolkata
62,56,Female,104.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi


**Without Column Transformer**

In [37]:
#adding simple imputer to fever 
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

#also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [44]:
#Ordinal encoding ->> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

#also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [42]:
#one hot encoding => gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

#also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])
X_train_gender_city.shape

(80, 4)

In [41]:
#extract age
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

#also extract from the test data
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_train_age.shape

(80, 1)

In [ ]:
#concatenate the transformed column
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis = 1)

#also the test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

X_train_transformed.shape 

(80, 7)

**With Column Transformation**

In [ ]:
from sklearn.compose import ColumnTransformer

In [52]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city'])], remainder='passthrough')

In [56]:
transformer.fit_transform(X_train).shape

(80, 7)

In [58]:
transformer.transform(X_test).shape

(20, 7)